#### Installing dependencies

In [ ]:
!sudo apt install octave

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  aglfn default-jre-headless epstool fonts-freefont-otf gnuplot-data
  gnuplot-qt info install-info libamd2 libauthen-sasl-perl libbtf1 libcamd2
  libccolamd2 libcholmod3 libclone-perl libcolamd2 libcombblas1.16.0
  libcxsparse3 libdata-dump-perl libemf1 libencode-locale-perl libevdev2
  libfftw3-long3 libfftw3-mpi3 libfftw3-single3 libfile-listing-perl
  libfltk-gl1.3 libfltk1.3 libfont-afm-perl libglpk40 libglu1-mesa
  libgraphicsmagick++-q16-12 libgraphicsmagick-q16-3 libgudev-1.0-0
  libhdf5-openmpi-103-1 libhtml-form-perl libhtml-format-perl
  libhtml-parser-perl libhtml-tagset-perl libhtml-tree-perl
  libhttp-cookies-perl libhttp-daemon-perl libhttp-date-perl
  libhttp-message-perl libhttp-negotiate-perl libhypre-2.22.1 libinput-bin
  libinput10 libio-html-perl libio-socket-ssl-perl libklu1 liblua5.4-0
  liblwp-mediatypes-perl liblw

In [ ]:
!pip install oct2py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.8/217.8 kB 7.9 MB/s eta 0:00:00


In [ ]:
!pip install torchmetrics

#### Importing pkgs

In [ ]:
import torch;
import torch.nn as nn;
import torch.nn.functional as F;
from torch.utils.data import Dataset, DataLoader;

import matplotlib.pyplot as plt;
import numpy as np;

from tqdm import tqdm;

In [ ]:
import os;
import os.path;

In [ ]:
import re;

In [ ]:
from oct2py import Oct2Py

oc = Oct2Py();

In [ ]:
from torchmetrics.classification import Accuracy,Precision, Recall, F1Score, ConfusionMatrix;

In [ ]:
import pandas as pd;
import random;

import json;

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu");
print(device);

cuda


In [ ]:
seed = 42;

#### Dataset Creation


In [ ]:
def getYLabel(filename):
    output = {
            "KA" : 1,
            "KI" : 0
        };

    code = filename.split("_")[3];

    if code.startswith("KA"):
        return output["KA"];
    else: return output["KI"];

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, code, healty_end_points, faulty_end_points, oc, width, signal="vibration"):
        self.root_dir = root_dir;
        self.healty_end_points = healty_end_points;
        self.faulty_end_points = faulty_end_points;
        self.oc = oc;

        self.width = width;

        self.X = [];
        self.Y = [];

        self.signals = {
            "vibration": 6,
            "current_1": 1,
            "current_2": 2,
        };

        self.signal = signal;

        for healty_end_point in self.healty_end_points:
            for mat_healthy_end_point in os.listdir(os.path.join(self.root_dir, healty_end_point)):
                if mat_healthy_end_point.endswith(".mat") and mat_healthy_end_point.startswith(code):
                    self.X.append(os.path.join(self.root_dir, healty_end_point, mat_healthy_end_point));
                    self.Y.append(getYLabel(mat_healthy_end_point));

        for faulty_end_point in self.faulty_end_points:
            for mat_faulty_end_point in os.listdir(os.path.join(self.root_dir, faulty_end_point)):
                if mat_faulty_end_point.endswith(".mat") and mat_faulty_end_point.startswith(code):
                    self.X.append(os.path.join(self.root_dir, faulty_end_point, mat_faulty_end_point));
                    self.Y.append(getYLabel(mat_faulty_end_point));

    def __len__(self):
        return len(self.X);

    def __getitem__(self, idx):
        data = self.oc.load(self.X[idx]);
        field = list(data.keys())[0];

        if self.signal == "vibration":
            x = data[field]['Y']['Data'][0][self.signals['vibration']].flatten();
            X = np.expand_dims(x, axis=0);
        elif self.signal == "current":
            x1 = data[field]['X']['Data'][0][self.signals['current_1']].flatten();
            x2 = data[field]['Y']['Data'][0][self.signals['current_2']].flatten();
            X = np.concatenate((x1, x2), axis=0);
        else:
            x = data[field]['Y']['Data'][0][self.signals['vibration']].flatten();
            x1 = data[field]['X']['Data'][0][self.signals['current_1']].flatten();
            x2 = data[field]['Y']['Data'][0][self.signals['current_2']].flatten();
            X = np.concatenate((x,x1,x2),axis=0);

        w = X.shape[-1];

        if(w < self.width):
            X = np.pad(X, pad_width = ((0,0),(0, self.width - w)), mode = 'constant');
        elif(w > self.width):
            X = X[:,:self.width];

        X = torch.from_numpy(X);
        X = X.to(torch.float32);
        y = torch.tensor(self.Y[idx]);


        X_min = torch.min(X,dim=-1);
        X_max = torch.max(X,dim=-1);

        X = (X - X_min.values) / (X_max.values - X_min.values);
        X = 2 * X - 1;

        return X, y;

In [ ]:
def getCode(N, M, F):
    return f"N{N:02}" + "_" + f"M{M:02}" + "_" + f"F{F:02}";

In [ ]:
# ratios = {2:5, 6:5, 8:5}
def getDataset(code, signal = "vibration", width=256000, ratios="6:5"):
    root_dir = "/kaggle/input/paderborn-db";
    healty_end_points = ["K001","K002","K003","K004","K005","K006"];
    faulty_end_points_inner = ["KI04","KI14","KI16","KI17","KI18","KI21"];
    faulty_end_points_outer = ["KA04","KA15","KA16","KA22","KA30"];

    f,h = ratios.split(":");

    f,h = int(f), int(h);

    healthy_training_end_points = [];
    healthy_testing_end_points = [];

    faulty_training_end_points = faulty_end_points_inner[:f//2] + faulty_end_points_outer[:f//2];
    faulty_testing_end_points = faulty_end_points_inner[f//2:] + faulty_end_points_outer[f//2:];

    train_dataset = CustomDataset(root_dir, code, healthy_training_end_points, faulty_training_end_points, oc, width, signal);
    test_dataset = CustomDataset(root_dir, code, healthy_testing_end_points, faulty_testing_end_points, oc, width, signal);

    return train_dataset, test_dataset;

In [ ]:
N = [9,15];
M = [7,1];
Fa = [10,4];

code = getCode(9,7,10);

In [ ]:
training_dataset, testing_dataset = getDataset(code,signal="vibration",width=256000,ratios="6:5");

In [ ]:
training_dataloader = DataLoader(training_dataset, batch_size=32, shuffle=True);
testing_dataloader = DataLoader(testing_dataset, batch_size=32, shuffle=True);

#### Model Creation

In [ ]:
import torch
import torch.nn as nn

class TransposeLayer(nn.Module):
    def __init__(self,dim1,dim2):
        super(TransposeLayer, self).__init__()

        self.dim1 = dim1;
        self.dim2 = dim2;

    def forward(self, x):
        return x.transpose(self.dim1, self.dim2);

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=7, stride=1, downsample=False):
        super(ResidualBlock, self).__init__()
        padding = kernel_size // 2

        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        self.ln1 = nn.LayerNorm(out_channels)
        self.relu = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, stride, padding, bias=False)
        self.ln2 = nn.LayerNorm(out_channels)

        self.downsample = nn.Sequential()
        if downsample or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                TransposeLayer(1,2),
                nn.LayerNorm(out_channels),
                TransposeLayer(1,2)
            )

    def forward(self, x):
        identity = self.downsample(x)

        out = self.conv1(x)
        out = out.transpose(1, 2)
        out = self.ln1(out)
        out = out.transpose(1, 2)
        out = self.relu(out)

        out = self.conv2(out)
        out = out.transpose(1, 2)
        out = self.ln2(out)
        out = out.transpose(1, 2)

        out += identity
        out = self.relu(out)

        return out

class ResNet1D_LN(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet1D_LN, self).__init__()
        self.input_layer = nn.Conv1d(1, 16, kernel_size=7, stride=2, padding=3, bias=False)
        self.ln = nn.LayerNorm(16)
        self.relu = nn.ReLU(inplace=True)
        self.pool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(16, 32, num_blocks=2, downsample=True)
        self.layer2 = self._make_layer(32, 64, num_blocks=2, downsample=True)
        self.layer3 = self._make_layer(64, 64, num_blocks=2, downsample=True)

        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)

        self.fc = nn.Linear(64, num_classes)

    def _make_layer(self, in_channels, out_channels, num_blocks, downsample):
        layers = [ResidualBlock(in_channels, out_channels, downsample=True)]
        for _ in range(1, num_blocks):
            layers.append(ResidualBlock(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.input_layer(x)
        x = x.transpose(1, 2)
        x = self.ln(x)
        x = x.transpose(1, 2)
        x = self.relu(x)
        x = self.pool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.global_avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [ ]:
model = ResNet1D_LN(num_classes=2)
model = model.to(device);

print(model);

ResNet1D_LN(
  (input_layer): Conv1d(1, 16, kernel_size=(7,), stride=(2,), padding=(3,), bias=False)
  (ln): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
  (relu): ReLU(inplace=True)
  (pool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv1d(16, 32, kernel_size=(7,), stride=(1,), padding=(3,), bias=False)
      (ln1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(32, 32, kernel_size=(7,), stride=(1,), padding=(3,), bias=False)
      (ln2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (downsample): Sequential(
        (0): Conv1d(16, 32, kernel_size=(1,), stride=(1,), bias=False)
        (1): TransposeLayer()
        (2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (3): TransposeLayer()
      )
    )
    (1): ResidualBlock(
      (conv1): Conv1d(32, 32, kernel_size=(7,), stride=(1,), padding=(

In [ ]:
loss = nn.CrossEntropyLoss();
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001);

In [ ]:
epochs = 75;

In [ ]:
accuracy = Accuracy(task="multiclass", num_classes=2).to(device);
precision = Precision(task="multiclass", num_classes=2).to(device);
recall = Recall(task="multiclass", num_classes=2).to(device);
f1_score = F1Score(task="multiclass", num_classes=2).to(device);
conf_matrix_metric = ConfusionMatrix(task="multiclass", num_classes=2).to(device);

In [ ]:
result_dir = "/kaggle/working/";
if not os.path.exists(result_dir):
    os.makedirs(result_dir);

#### Model Training

In [ ]:
metrics_list = [];

for epoch in range(epochs):

    train_loss_per_epoch = [];
    test_loss_per_epcoh = [];

    train_acc_per_epoch = [];
    test_acc_per_epoch = [];

    train_precision_per_epoch = [];
    test_precision_per_epoch = [];

    train_recall_per_epoch = [];
    test_recall_per_epoch = [];

    train_f1_per_epoch = [];
    test_f1_per_epoch = [];

    traing_cf = np.zeros((2,2));
    test_cf = np.zeros((2,2));

    model.train();
    for x, y in training_dataloader:
        optimizer.zero_grad();
        x = x.to(device);
        y = y.to(device);

        y_pred = model(x);
        loss_value = loss(y_pred, y);

        loss_value.backward();
        optimizer.step();

        acc = accuracy(y_pred, y);
        prec = precision(y_pred, y);
        rec = recall(y_pred, y);
        f1 = f1_score(y_pred, y);
        conf_matrix = conf_matrix_metric(y_pred, y);

        traing_cf += conf_matrix.cpu().numpy();

        train_loss_per_epoch.append(loss_value.item());
        train_acc_per_epoch.append(acc.item());
        train_precision_per_epoch.append(prec.item());
        train_recall_per_epoch.append(rec.item());
        train_f1_per_epoch.append(f1.item());

    model.eval();
    with torch.no_grad():
        for x, y in testing_dataloader:
            x = x.to(device);
            y = y.to(device);

            y_pred = model(x);
            loss_value = loss(y_pred, y);

            acc = accuracy(y_pred, y);
            prec = precision(y_pred, y);
            rec = recall(y_pred, y);
            f1 = f1_score(y_pred, y);
            conf_matrix = conf_matrix_metric(y_pred, y);

            test_cf += conf_matrix.cpu().numpy();

            test_loss_per_epcoh.append(loss_value.item());
            test_acc_per_epoch.append(acc.item());
            test_precision_per_epoch.append(prec.item());
            test_recall_per_epoch.append(rec.item());
            test_f1_per_epoch.append(f1.item());


    metrics_list.append({
        "train_loss": np.mean(train_loss_per_epoch),
        "test_loss": np.mean(test_loss_per_epcoh),
        "train_acc": np.mean(train_acc_per_epoch),
        "test_acc": np.mean(test_acc_per_epoch),
        "train_precision": np.mean(train_precision_per_epoch),
        "test_precision": np.mean(test_precision_per_epoch),
        "train_recall": np.mean(train_recall_per_epoch),
        "test_recall": np.mean(test_recall_per_epoch),
        "train_f1": np.mean(train_f1_per_epoch),
        "test_f1": np.mean(test_f1_per_epoch),
        "train_cf": json.dumps(traing_cf.tolist()),
        "test_cf": json.dumps(test_cf.tolist())
    });

    maxx = max(metrics_list, key=lambda x: x["test_f1"]);

    if maxx["test_f1"] <= metrics_list[-1]["test_f1"]:
        torch.save(model.state_dict(), os.path.join(result_dir,"bb_mm.pth"));

    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {metrics_list[-1]['train_loss']:.4f} | Test Loss: {metrics_list[-1]['test_loss']:.4f} | Train Acc: {metrics_list[-1]['train_acc']:.4f} | Test Acc: {metrics_list[-1]['test_acc']:.4f} | Train F1: {metrics_list[-1]['train_f1']:.4f} | Test F1: {metrics_list[-1]['test_f1']:.4f}");

Epoch 1/75 | Train Loss: 0.7184 | Test Loss: 0.7260 | Train Acc: 0.5208 | Test Acc: 0.5312 | Train F1: 0.5208 | Test F1: 0.5312
Epoch 2/75 | Train Loss: 0.6686 | Test Loss: 0.7508 | Train Acc: 0.5990 | Test Acc: 0.4453 | Train F1: 0.5990 | Test F1: 0.4453
Epoch 3/75 | Train Loss: 0.6504 | Test Loss: 0.6965 | Train Acc: 0.6458 | Test Acc: 0.6016 | Train F1: 0.6458 | Test F1: 0.6016
Epoch 4/75 | Train Loss: 0.6201 | Test Loss: 0.7386 | Train Acc: 0.7031 | Test Acc: 0.5703 | Train F1: 0.7031 | Test F1: 0.5703
Epoch 5/75 | Train Loss: 0.6237 | Test Loss: 0.6672 | Train Acc: 0.7031 | Test Acc: 0.6172 | Train F1: 0.7031 | Test F1: 0.6172
Epoch 6/75 | Train Loss: 0.6196 | Test Loss: 0.6982 | Train Acc: 0.6823 | Test Acc: 0.5625 | Train F1: 0.6823 | Test F1: 0.5625
Epoch 7/75 | Train Loss: 0.5930 | Test Loss: 0.6560 | Train Acc: 0.7057 | Test Acc: 0.6406 | Train F1: 0.7057 | Test F1: 0.6406
Epoch 8/75 | Train Loss: 0.6088 | Test Loss: 0.6962 | Train Acc: 0.6953 | Test Acc: 0.5859 | Train F1: 0

#### Storing Results

In [ ]:
df_epochs = pd.DataFrame(metrics_list);
df_epochs.to_csv(os.path.join(result_dir,"result.csv"),index=False);